In [1]:
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
from operator import itemgetter
import warnings
import pickle
from Bio import Seq
from Bio import SeqIO
import pysam
import itertools
import scipy
from scipy import stats
from scipy.interpolate import interp1d
from scipy.stats import chi2_contingency
import statsmodels.formula.api as smf
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
warnings.filterwarnings("ignore")


Bad key "text.kerning_factor" on line 4 in
/lustre/user/lulab/doushq/wuxk/software/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
ErrorRate_df = pd.read_csv("../Results/DatasetS2.csv",sep="\t")
ErrorRate_df.rename(columns={ErrorRate_df.columns[0]: 'codon'}, inplace=True)
ErrorRate_df.head(2)

,codon,embryo_0-2h_1,embryo_0-2h_2,embryo_0-2h_3,embryo_0-2h_4,embryo_10-12h_1,embryo_10-12h_2,embryo_10-12h_3,embryo_10-12h_4,embryo_18-20h_1,...,pupae_day5_3,pupae_day5_4,virgin_female_1,virgin_female_2,virgin_female_3,virgin_female_4,virgin_male_1,virgin_male_2,virgin_male_3,virgin_male_4
0,TTT,0.000949,0.000000,0.002160,0.000000,0.001580,0.000060,0.000043,0.000064,0.000009,...,0.000038,0.000044,0.000008,0.000008,0.000014,0.000050,0.0,0.000012,0.000018,0.000017
1,TTC,0.000025,0.000009,0.000107,0.000155,0.000077,0.000014,0.000043,0.000011,0.000086,...,0.000005,0.000206,0.000077,0.000000,0.000007,0.000002,0.0,0.000017,0.000000,0.000009


In [3]:
codon_index_df = pd.read_csv("../Data/CodonIndex_Flybase.csv",sep="\t")
codon_index_df.rename(columns={codon_index_df.columns[0]: 'codon'}, inplace=True)
codon_index_df.head(2)

,codon,CodonUsage,RSCU,category
0,TTT,947386.0,0.492734,nonoptimal
1,TTC,2898039.0,1.507266,optimal


### Rate and RSCU

In [4]:
errorRate_samples_melt = pd.melt(ErrorRate_df,id_vars='codon',value_vars=ErrorRate_df.columns[1:],var_name='sample',value_name='errorRate')
errorRate_samples_melt.columns = ['codon','sample','errorRate']
errorRate_samples_melt['replicate'] = errorRate_samples_melt['sample'].apply(lambda x: '_'.join(x.split('_')[-1]))
errorRate_samples_melt['condition'] = errorRate_samples_melt['sample'].apply(lambda x: '_'.join(x.split('_')[:-1]))
errorRate_samples_melt['log10errorRate'] = np.log10(errorRate_samples_melt['errorRate'])
errorRate_samples_melt = pd.merge(errorRate_samples_melt,codon_index_df,on='codon')
errorRate_samples_melt.head(2)

,codon,sample,errorRate,replicate,condition,log10errorRate,CodonUsage,RSCU,category
0,TTT,embryo_0-2h_1,0.000949,1,embryo_0-2h,-3.022734,947386.0,0.492734,nonoptimal
1,TTT,embryo_0-2h_2,0.000000,2,embryo_0-2h,-inf,947386.0,0.492734,nonoptimal


In [5]:
errorRate_samples_melt = errorRate_samples_melt[errorRate_samples_melt['errorRate']>0]
errorRate_samples_melt.head(2)

,codon,sample,errorRate,replicate,condition,log10errorRate,CodonUsage,RSCU,category
0,TTT,embryo_0-2h_1,0.000949,1,embryo_0-2h,-3.022734,947386.0,0.492734,nonoptimal
2,TTT,embryo_0-2h_3,0.002160,3,embryo_0-2h,-2.665546,947386.0,0.492734,nonoptimal


In [6]:
model = smf.mixedlm("log10errorRate~RSCU+condition", errorRate_samples_melt, groups=errorRate_samples_melt["replicate"])
result = model.fit()

print(result.summary())

                Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    log10errorRate
No. Observations:     2809       Method:                REML          
No. Groups:           4          Scale:                 0.4757        
Min. group size:      695        Log-Likelihood:        -2972.9335    
Max. group size:      708        Converged:             No            
Mean group size:      702.2                                           
----------------------------------------------------------------------
                           Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                  -4.004    0.063 -63.833 0.000 -4.127 -3.881
condition[T.embryo_0-2h]    0.533    0.072   7.372 0.000  0.391  0.674
condition[T.embryo_10-12h]  0.110    0.073   1.516 0.130 -0.032  0.253
condition[T.embryo_18-20h]  0.029    0.072   0.397 0.691 -0.112  0.169
condition[T.embryo_4-6h

### RMR and RSCU

In [7]:
errorRate_samples_melt.head(2)

,codon,sample,errorRate,replicate,condition,log10errorRate,CodonUsage,RSCU,category
0,TTT,embryo_0-2h_1,0.000949,1,embryo_0-2h,-3.022734,947386.0,0.492734,nonoptimal
2,TTT,embryo_0-2h_3,0.002160,3,embryo_0-2h,-2.665546,947386.0,0.492734,nonoptimal


In [8]:
errorRate_melt = errorRate_samples_melt.copy()
errorRate_melt['protein'] = errorRate_melt['codon'].apply(lambda x: str(Seq.Seq(x).translate()))

average_rate = errorRate_melt.groupby(['protein','sample'])['errorRate'].mean().reset_index()
average_rate.columns = ['protein','sample','meanRate']

errorRate_melt_mean = pd.merge(errorRate_melt,average_rate,on=['protein','sample'])
errorRate_melt_mean['RMR'] = errorRate_melt_mean['errorRate']/errorRate_melt_mean['meanRate']
errorRate_melt_mean['UMI'] = errorRate_melt_mean['sample'] + errorRate_melt_mean['protein']

# remove amino acids without detection
remove_sites = errorRate_melt[errorRate_melt['errorRate'] == 0]
remove_sites['UMI'] = remove_sites['sample'] + remove_sites['protein']

errorRate_melt_mean_clean = errorRate_melt_mean[~errorRate_melt_mean['UMI'].isin(remove_sites['UMI'])]
del errorRate_melt_mean_clean['UMI']
errorRate_melt_mean_clean.head(2)

,codon,sample,errorRate,replicate,condition,log10errorRate,CodonUsage,RSCU,category,protein,meanRate,RMR
0,TTT,embryo_0-2h_1,0.000949,1,embryo_0-2h,-3.022734,947386.0,0.492734,nonoptimal,F,0.000487,1.948265
1,TTC,embryo_0-2h_1,0.000025,1,embryo_0-2h,-4.598599,2898039.0,1.507266,optimal,F,0.000487,0.051735


In [9]:
removed_protein = ['W','M']
errorRate_melt_mean_clean = errorRate_melt_mean_clean[~errorRate_melt_mean_clean['protein'].isin(removed_protein)]

In [10]:
model = smf.mixedlm("RMR~RSCU+condition", errorRate_melt_mean_clean, groups=errorRate_melt_mean_clean["replicate"])
result = model.fit()

print(result.summary())

                Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     RMR       
No. Observations:      2722        Method:                 REML      
No. Groups:            4           Scale:                  0.6208    
Min. group size:       670         Log-Likelihood:         -3242.6104
Max. group size:       687         Converged:              No        
Mean group size:       680.5                                         
---------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                   1.157    0.081 14.225 0.000  0.998  1.317
condition[T.embryo_0-2h]    0.006    0.084  0.069 0.945 -0.159  0.171
condition[T.embryo_10-12h]  0.007    0.084  0.086 0.932 -0.158  0.173
condition[T.embryo_18-20h]  0.003    0.083  0.036 0.972 -0.161  0.166
condition[T.embryo_4-6h]    0.008   

### keep zero

In [11]:
ErrorRate_melt = pd.melt(ErrorRate_df,id_vars='codon',value_vars=ErrorRate_df.columns[1:])
ErrorRate_melt.columns = ['codon','sample','errorRate']
ErrorRate_melt_codon = pd.merge(ErrorRate_melt,codon_index_df,on='codon')
ErrorRate_melt_codon['protein'] = ErrorRate_melt_codon['codon'].apply(lambda x: str(Seq.Seq(x).translate()))
ErrorRate_melt_codon.head(2)

,codon,sample,errorRate,CodonUsage,RSCU,category,protein
0,TTT,embryo_0-2h_1,0.000949,947386.0,0.492734,nonoptimal,F
1,TTT,embryo_0-2h_2,0.000000,947386.0,0.492734,nonoptimal,F


In [12]:
ErrorRate_melt_category = ErrorRate_melt_codon.groupby(['sample','protein','category'])['errorRate'].mean().reset_index()
ErrorRate_melt_category.head(2)

,sample,protein,category,errorRate
0,crawling_larvae_1,A,nonoptimal,0.000280
1,crawling_larvae_1,A,optimal,0.000243


In [13]:
removed_protein = ['W','M']
ErrorRate_melt_category = ErrorRate_melt_category[~ErrorRate_melt_category['protein'].isin(removed_protein)]
ErrorRate_melt_category.head(2)

,sample,protein,category,errorRate
0,crawling_larvae_1,A,nonoptimal,0.000280
1,crawling_larvae_1,A,optimal,0.000243


In [14]:
ErrorRate_melt_category['condition'] = ErrorRate_melt_category['sample'].apply(lambda x: '_'.join(x.split('_')[:-1]))
ErrorRate_melt_category['replicate'] = ErrorRate_melt_category['sample'].apply(lambda x: x.split('_')[-1])
ErrorRate_melt_category.head(2)

,sample,protein,category,errorRate,condition,replicate
0,crawling_larvae_1,A,nonoptimal,0.000280,crawling_larvae,1
1,crawling_larvae_1,A,optimal,0.000243,crawling_larvae,1


In [15]:
model = smf.mixedlm("errorRate~category+condition+protein", ErrorRate_melt_category, groups=ErrorRate_melt_category["replicate"])
result = model.fit()

print(result.summary())

                Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     errorRate 
No. Observations:      2448        Method:                 REML      
No. Groups:            4           Scale:                  0.0000    
Min. group size:       612         Log-Likelihood:         11086.9502
Max. group size:       612         Converged:              No        
Mean group size:       612.0                                         
---------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                   0.000    0.000  0.251 0.802 -0.001  0.001
category[T.optimal]        -0.000    0.000 -2.158 0.031 -0.000 -0.000
condition[T.embryo_0-2h]    0.001    0.000  5.123 0.000  0.001  0.002
condition[T.embryo_10-12h]  0.000    0.000  0.220 0.826 -0.000  0.001
condition[T.embryo_18-20h]  0.000   